C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Clienti per area territoriale\Clienti diretta\NordEst

In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt

from os import listdir
from os.path import isfile, join
from os.path import getmtime

from zebra import remov_duplicates, human_format
from babel.numbers import format_number

In [2]:
idx = pd.IndexSlice

In [22]:
pd.set_option('display.max_rows', 100)

# Sources

## Parametriche

In [3]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)

In [6]:
par.keys()

dict_keys(['Agente_ordine', 'Area_prodotto', 'Canale_vendita', 'Classificazione_codice_prodotto - New', 'Classificazione_codice_prodotto', 'Classificazione_mdm', 'Clienti_inside_sales', 'Clienti_inside_sales_nuovi_2021', 'Esclusione_fatture', 'Esclusione_ordini', 'Forfait', 'Giorni_lavorativi', 'Linea_prodotti', 'Opportunity_owner', 'Sales_director'])

In [4]:
cod_prod = par["Classificazione_codice_prodotto"]
cod_prod["Codice PRODOTTO"] = cod_prod["Codice PRODOTTO"].apply(lambda x: str(x))
cod_prod = cod_prod.loc[:, ['MDM', 'GTM BDG', 'Solution','Sub solution',"Codice PRODOTTO"]]

sales_director = par["Sales_director"]
sales_director = sales_director.loc[:, ["Sales Director", "Area territoriale", "Rete"]]
sales_director["Sales Director"] = sales_director["Sales Director"].str.lower()

agente_ordine = par["Agente_ordine"]

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

li_prodotti = par["Linea_prodotti"]

## Parco clienti da Licenze e abbonamenti

In [5]:
clienti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Diretta Indiretta IndirettaWKInfo\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Diretta Indiretta IndirettaWKInfo") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Parco clienti\Data export\Diretta Indiretta IndirettaWKInfo", f))]

dfs = []
for file in onlyfiles:
    data = pd.read_excel(clienti+file, na_values="#", dtype={"Data inizio contratt":object, "Data Inizio Abb.":object, "Data Fine Abb.":object, "Cliente FATTURA":str, "Codice ABBONAMENTO":str})
    data["Tipo Licenza"] = file.split("_")[1].split(".")[0]
    data["Ultimo Agg."] = dt.date.fromtimestamp(getmtime(clienti+file))
    dfs.append(data)

cli = pd.concat(dfs, axis=0, ignore_index=True)
    
for col in cli.columns:
    if "Data" in col:
        
        cli[col] = cli[col].apply(lambda x: (float(x) - 25569) * 86400)
        cli[col] = pd.to_datetime(cli[col], unit="s")
    else:
        pass

cli.rename(columns={'Unnamed: 10':'Descrizione ABBONAMENTO'}, inplace=True )

## All accounts da Salesforce

In [22]:
accounts_sf = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_all_accounts.csv", encoding='latin-1', sep=";")

In [69]:
accounts_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97881 entries, 0 to 97880
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   WK Account Number  95495 non-null  object
 1   Account Owner      97881 non-null  object
 2   Account Name       97881 non-null  object
 3   Type               97869 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


---

# Lista di clienti con determinati prodotti

## Filtri

Inserire valore per stato abbonamento e per stato contratto

In [7]:
stato_abb = (cli["Stato Esteso"].str.contains("ATTIVO")) & (cli["Stato Sintetico Contratto"].isin(["Attivo","Sospeso"]))

Inserire colonne da considerare

In [8]:
columns = ["Cliente MERCE","Segmento Aggregato Cl.Merce","Agente dell'ORDINE","Rete", "Area territoriale", 'Provincia', 'Solution', 'Sub solution', "Sistema","Fornitura (SW)","Valore PV New"]

#'Cliente MERCE', 'Segmento Aggregato Cl.Merce', 'Regione', 'Provincia','Canale di VENDITA', 'Cliente FATTURA', 'Numero Contratto Lic','Stato Sintetico Contratto', 'Data inizio contratt','Codice ABBONAMENTO', 'Unnamed: 10', 'Soluzione (SW)', 'Prodotto (SW)','Cluster (SW)', 'Sistema', 'Fornitura (SW)', "Agente dell'Abbonamento",'Sales Director', 'Stato Esteso', 'Data Inizio Abb.', 'Data Fine Abb.','Valore PV New', 'Tipo Licenza', 'Ultimo Agg.'
#'MDM', 'GTM BDG', 'Solution', 'Sub solution', 'Codice PRODOTTO'
#'Sales Director', 'Area territoriale', 'Rete'

Inserire area territoriale

In [9]:
area_territ = ['Nord-Est']

#'Enterprise indiretta', 'Rete IPSOA', 'Large accounts', 'Enterprise', 'Tax&Pay indiretta', 'Rete WKI', 'Centro-Sud', 'Nord-Est', 'Nord-Ovest'

## Creazione dataframe filtrato

In [23]:
clienti = cli.loc[stato_abb, :].copy()

In [24]:
clienti["Sales Director"] = clienti["Sales Director"].str.lower()
clienti["Codice ABBONAMENTO"] = clienti.loc[:,"Codice ABBONAMENTO"].apply(str)

In [25]:
def cli_prod_saldir_ag(df_):
    df = clienti.merge(cod_prod, how="left", left_on="Codice ABBONAMENTO", right_on="Codice PRODOTTO")\
          .merge(sales_director, how="left", on="Sales Director")\
          .merge(agente_ordine, how="left", left_on="Agente dell'Abbonamento", right_on="Codice agente")
    df["Cliente MERCE"] = df["Cliente MERCE"].apply(str)
    
    return df.loc[df["Area territoriale"].isin(area_territ), columns]

In [26]:
clienti = clienti.pipe(cli_prod_saldir_ag)

## Creazione nuove colonne aggregate

In [27]:
in_col = lambda col, string: clienti[col].str.contains(string, na=False)

conds = {'Studi commercialisti': in_col('Segmento Aggregato Cl.Merce', 'Fiscal|Studi Multi'),
         'Studi lavoro': in_col('Segmento Aggregato Cl.Merce', 'Lavoro'),
         'Aziende': in_col('Segmento Aggregato Cl.Merce', 'Aziende'),
         'Altro': ~in_col('Segmento Aggregato Cl.Merce', 'Fiscal|Studi Multi|Lavoro|Aziende')}

clienti["Tipologia cliente"] = np.select(condlist=conds.values(), choicelist=conds.keys())

## Modifica valore Solution per comprendere uno o più valori della Sub Solution

In [28]:
clienti.loc[(clienti["Sub solution"] == "Tuttotel FE"),  "Solution"] = "Tuttotel FE"
clienti.loc[(clienti["Sub solution"] == "DIGY EC"),  "Solution"] = "DIGY EC"

## Creazione cluster prodotti

In [29]:
def concat_prodotti(df_):
    df = clienti.loc[:,["Solution", "Cliente MERCE"]]
    df["Solution"] = pd.Categorical(df["Solution"], np.sort(df["Solution"].unique()))
    df.sort_values(by=["Solution"], inplace=True)
    df["Prodotti"] = df.groupby("Cliente MERCE")["Solution"].transform(lambda x: " + ".join(x)).apply(lambda x: remov_duplicates(x))
    return df["Prodotti"]

In [30]:
clienti = clienti.assign(Prodotti = lambda df_: df_.pipe(concat_prodotti))

## Aggiunta della ragione sociale

In [31]:
clienti["Codice SAP"] = clienti["Cliente MERCE"].astype(str).apply(lambda x: "IT-" + x.zfill(10))
clienti = clienti.merge(accounts_sf, how="left", left_on="Codice SAP", right_on="WK Account Number")

## Ultima definizione colonne

In [32]:
clienti = clienti.loc[:,["Cliente MERCE", "Account Name", "Tipologia cliente", "Agente dell'ORDINE", "Area territoriale","Provincia", "Fornitura (SW)", "Prodotti", "Sistema", "Solution","Sub solution", "Valore PV New"]].copy()
clienti.rename(columns={"Valore PV New":"Valore PV", "Agente dell'ORDINE":"Agente"}, inplace=True)

## Export in excel

In [33]:
path_base_dati = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Analisi clienti\Clienti per area territoriale\Clienti diretta\NordEst\Data export\\"

In [34]:
clienti.to_excel(path_base_dati+"clienti_NordEst.xlsx", index=False)